# 到YAHOO財經將今日的交易資料取下，並建立成資料。

In [45]:
def get_min_time_difference(driver):
    # 使用 Selenium 獲取動態生成的網頁內容
    page_source = driver.page_source

    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # 找到所有的 table 行
    table_rows = soup.find_all('li', class_='List(n)')

    times = []

    # 遍歷每一行並提取所需的信息
    for row in table_rows:
        # 提取時間
        time_div = row.find('div', class_='Fxs(0) Mend($m-table-cell-space) Ta(start) W(72px)')
        time = time_div.text.strip() if time_div else 'N/A'
        times.append(time)

    # 將時間數據轉換為 DataFrame
    df = pd.DataFrame(times, columns=['時間'])

    # 去除 "N/A" 值
    df = df[df['時間'] != 'N/A']

    return df.iloc[-1]


預計完成日：5/15 ~　5/17
預計步驟
 1. 到YAHOO財經網站
 2. 到該股票的成交彙整，點擊載入更多
 3. 取下資料，存成 股票代號_名稱_今日日期


In [88]:
# 操作 browser 的 API 
from selenium.webdriver.chrome.service import Service 
from selenium import webdriver 
# 匯入套件 
from bs4 import BeautifulSoup as bs 
# 強制等待 (執行期間休息一下) 
from time import sleep
# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")        #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

import pyautogui
import random
from pyautogui import ImageNotFoundException

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [89]:
my_service = Service(executable_path="./chromedriver-win64/chromedriver.exe") 
driver = webdriver.Chrome(options = my_options,service=my_service)

In [51]:
t_end_pd
# 將 DataFrame 存成 CSV 文件
t_end_pd.to_csv('t_end_pd_0519.csv', index=False, encoding='utf-8-sig')

In [95]:
stockNo_str = random.choice([2454, 2317, 2382, 2303, 2357])
url = f"https://tw.stock.yahoo.com/quote/{stockNo_str}/time-sales"
driver.get(url)

# 睡個幾秒
sleep(2)


# 創建一個DataFrame包含t_end
t_end_pd = pd.DataFrame()

# 等待廣告關閉按鈕出現
try:
    target_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[target="_blank"]'))  # 根據你的 HTML 片段
    )

    # # 等待一段時間，確保廣告完全關閉
    # sleep(5)
    
    # 進行後續操作，例如截圖或模擬點擊
    advertise_close_button = pyautogui.locateOnScreen('yahoo_advertise_close.png', confidence=0.9)
    # 取得每一個圖片的中心點
    advertise_close_button_point = pyautogui.center(advertise_close_button)
    pyautogui.click(advertise_close_button_point)
    
finally:
    # 睡個幾秒
    sleep(2)

click_count = 0
last_time = 0
scroll_count = 0  # 初始化滾動次數計數器
max_scroll_count = 240  # 設置滾動次數限制
refresh_count = 0

while scroll_count < max_scroll_count:
    # 向下滾動
    pyautogui.scroll(-600)
    sleep(1)  # 等待一秒，讓頁面加載

    # 檢查是否找到 load_more_button
    try:
        load_more_button = pyautogui.locateOnScreen('yahoo_load_more.png', confidence=0.9)
        if load_more_button:
            # 找到按鈕，點擊它
            load_more_button_point = pyautogui.center(load_more_button)
            pyautogui.click(load_more_button_point)
            click_count += 1
            
            # 在每次點擊後調用函數，並將結果加入到 DataFrame 中
    except ImageNotFoundException:
        pass
    
    # 如果滾動次數超過點擊次數的五次以上，則打印點擊次數並退出迴圈
    if last_time == click_count:
        print(f"總共點擊 {click_count} 次")
        print(f"總共滾動 {scroll_count} 次")
        break
    else:  
        try:
            data_loading_error_button = pyautogui.locateOnScreen('yahoo_data_loading_error.png', confidence=0.9)
        except ImageNotFoundException:
            # 重新整理頁面
            pyautogui.hotkey('ctrl', 'r')
            refresh_count += 1
            # 更新參數
            click_count = 0
            scroll_count = 0
            max_scroll_count = 240
            print(f"重新整理 {refresh_count} 次")
            sleep(2)
            # 進行後續操作，例如截圖或模擬點擊
            advertise_close_button = pyautogui.locateOnScreen('yahoo_advertise_close.png', confidence=0.9)
            # 取得每一個圖片的中心點
            advertise_close_button_point = pyautogui.center(advertise_close_button)
            pyautogui.click(advertise_close_button_point)

         

    scroll_count += 1
    # 增加滾動次數計數器

    page_source = driver.page_source
    t_end = get_min_time_difference(driver)
    t_end_pd = pd.concat([t_end_pd, pd.DataFrame({"時間": [t_end]})], ignore_index=True)

    last_time = click_count



# 使用 Selenium 獲取動態生成的網頁內容
page_source = driver.page_source

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(page_source, 'html.parser')

#  找到所有的 table 行
table_rows = soup.find_all('li', class_='List(n)')

data = []

# 遍歷每一行並提取所需的信息
for row in table_rows:
    # 提取時間
    time_div = row.find('div', class_='Fxs(0) Mend($m-table-cell-space) Ta(start) W(72px)')
    time = time_div.text.strip() if time_div else 'N/A'
    
    # 提取成交價
    price_span = row.find('span', class_='Jc(fe) Fw(600) D(f) Ai(c)')  # 第一種 class
    price_span_up = row.find('span', class_='Jc(fe) Fw(600) D(f) Ai(c) C($c-trend-up)')  # 第二種 class
    price_span_down = row.find('span', class_='Jc(fe) Fw(600) D(f) Ai(c) C($c-trend-down)')  # 第三種 class

    if price_span:
        price = price_span.text.strip().replace(',', '')
    elif price_span_up:
        price = price_span_up.text.strip().replace(',', '')
    elif price_span_down:
        price = price_span_down.text.strip().replace(',', '')
    else:
        price = 'N/A'
    
    # 提取成交張數
    volume_divs = row.find_all('div', class_='Fxs(0) Fxg(1) Fxb(66px) Ta(end) Mend(12px) Mend(0):lc')
    if len(volume_divs) > 0:
        volume = volume_divs[-1].text.strip().replace('"', '').replace(',', '')
    else:
        volume = 'N/A'
    
    data.append([time, price, volume])

# 將數據放入 DataFrame
df = pd.DataFrame(data, columns=['時間', '成交價', '成交張數'])

# 將"N/A"替換為NaN
df.replace("N/A", np.nan, inplace=True)

#  如果 ['時間', '成交價', '成交張數'] 同時為 NaN，則刪除該行
df = df.dropna(subset=['時間', '成交價', '成交張數'], how='all')

columns_to_convert_float = ['時間', '成交價', '成交張數']
for column in columns_to_convert_float:
    if column == '時間':
        df[column] = df[column].astype('datetime64[ns]')  # 轉換為 datetime64 型態
    elif column == '成交張數':
        # 將成交張數轉換為浮點數型態，然後四捨五入至整數位
        df[column] = round(df[column].astype(float)).astype(float)
    else:
        # 針對其他列進行浮點數轉換，取小數點後兩位
        df[column] = round(df[column].astype(float), 1)

# # 创建一个包含完整时间范围的时间序列
# full_time_range = pd.date_range(start=df["時間"].max(), end=df["時間"].min(), freq='-5s')

# # 將時間範圍設置為datetime64格式
# full_time_range = full_time_range.astype('datetime64[ns]')

# # 使用reindex方法重新索引你的DataFrame
# df_change = df.set_index("時間").reindex(full_time_range).reset_index().rename(columns={"index": "時間"})

# # 檢查最後一行是否為NaN
# if pd.isna(df_change.loc[df_change.index[-1], "成交價"]):
#     # 找到倒數第二行的非NaN值
#     last_valid_index = df_change["成交價"].last_valid_index()
#     if last_valid_index is not None:
#         df_change.loc[df_change.index[-1], "成交價"] = df_change.loc[last_valid_index, "成交價"]
        
# # 使用方法ffill()來填補缺失值，向上填補最近的非空值
# df_change["成交價"] = df_change["成交價"].fillna(method='bfill')

# df_change['成交張數'] = df_change['成交張數'].fillna(0)

# 將 DataFrame 存成 CSV 文件
# df_change.to_csv('stock_data_tset_0519.csv', index=False, encoding='utf-8-sig')
df.to_csv('stock_data_tset_0519_沒有轉時間.csv', index=False, encoding='utf-8-sig')

# print(df)

# 關閉瀏覽器
# driver.quit()


總共點擊 77 次
總共滾動 77 次


C:\Users\D\AppData\Local\Temp\ipykernel_17008\496373148.py:144: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = df[column].astype('datetime64[ns]')  # 轉換為 datetime64 型態


In [84]:
df

,時間,成交價,成交張數
0,2024-05-19 13:30:00,"1,165",660
1,2024-05-19 13:24:55,"1,170",1
2,2024-05-19 13:24:45,"1,170",1
3,2024-05-19 13:24:35,"1,170",4
4,2024-05-19 13:24:30,"1,170",8
...,...,...,...
1184,2024-05-19 09:01:00,"1,180",1
1185,2024-05-19 09:00:50,"1,175",2
1186,2024-05-19 09:00:40,"1,175",2
1187,2024-05-19 09:00:30,"1,175",29


In [94]:
try:
    data_loading_error_button = pyautogui.locateOnScreen('yahoo_data_loading_error.png', confidence=0.9)
    print("123")
except ImageNotFoundException:
    print("test")

test


In [76]:
# data_loading_error_button = pyautogui.locateOnScreen('yahoo_data_loading_error.png', confidence=0.9)

import pyautogui
from pyautogui import ImageNotFoundException

try:
    data_loading_error_button = pyautogui.locateOnScreen('yahoo_data_loading_error.png', confidence=0.9)
except ImageNotFoundException:
    print("找不到 yahoo_data_loading_error.png")

找不到 yahoo_data_loading_error.png


In [15]:
merged_df = pd.concat([df], ignore_index=True)
# 檢查每個資料格的資料格式
different_formats = {}

for col in merged_df.columns:
    formats = merged_df[col].apply(type).unique()
    if len(formats) > 1:
        different_formats[col] = formats

if different_formats:
    print("下列格子的資料格式不同：")
    for col, formats in different_formats.items():
        print(f"列名: {col}, 不同的資料格式: {formats}")
else:
    print("所有格子的資料格式相同：")
    for col in merged_df.columns:
        print(f"列名: {col}, 資料格式: {merged_df[col].dtype}")

所有格子的資料格式相同：
列名: 時間, 資料格式: datetime64[ns]
列名: 成交價, 資料格式: float64
列名: 成交張數, 資料格式: float64


In [22]:

df_change
# 將 DataFrame 存成 CSV 文件
df_change.to_csv('stock_data_tset.csv', index=False, encoding='utf-8-sig')

# 畫圖

In [14]:
# 讀取 CSV 檔案
file_path = r'C:\Users\D\Desktop\project\股票程式碼\stock_data_tset.csv'
df = pd.read_csv(file_path)
# df["時間"]

columns_to_convert_float = ['時間', '成交價', '成交張數']
for column in columns_to_convert_float:
    if column == '時間':
        df[column] = df[column].astype('datetime64[ns]')  # 轉換為 datetime64 型態
    elif column == '成交張數':
        # 將成交張數轉換為浮點數型態，然後四捨五入至整數位
        df[column] = round(df[column].astype(float)).astype(float)
    else:
        # 針對其他列進行浮點數轉換，取小數點後兩位
        df[column] = round(df[column].astype(float), 1)

In [29]:
import matplotlib.pyplot as plt
def resample_data(df, time_interval):
    """
    對DataFrame進行重新取樣，計算在給定時間間隔內的開盤價、收盤價、最高價、最低價和成交張數。
    
    參數:
        df (DataFrame): 要重新取樣的原始數據。
        time_interval (str): 指定的時間間隔，如'5T'表示5分鐘間隔。
        
    返回:
        DataFrame: 重新取樣後的數據。
    """
    resampled = df.resample(time_interval, on='時間').agg({
        '成交價': ['first', 'last', 'max', 'min'],
        '成交張數': 'sum'
    })
    resampled.columns = ['開盤價', '收盤價', '最高價', '最低價', '成交張數']
    resampled.reset_index(inplace=True)
    return resampled


def plot_ohlc(data, title):
    """
    根據給定的開盤價、收盤價、最高價和最低價數據繪製 OHLC 圖表。

    參數:
        data (DataFrame): 包含開盤價、收盤價、最高價和最低價數據的 DataFrame。
        title (str): 圖表的標題。
    """
    plt.figure(figsize=(14, 7))
    plt.plot(data['時間'], data['開盤價'], label='Open')
    plt.plot(data['時間'], data['收盤價'], label='Close')
    plt.plot(data['時間'], data['最高價'], label='High')
    plt.plot(data['時間'], data['最低價'], label='Low')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

# 計算 5 分線
data_5min = resample_data(df_change, '5T')
data_5min.head()
# plot_ohlc(df_change, '5T')

# # Plot 30-minute line
# data_30min = resample_data(df_change, '30T')
# plot_ohlc(data_30min, '30-Minute Line')

,時間,開盤價,收盤價,最高價,最低價,成交張數
0,2024-05-16 09:00:00,174.5,178.5,179.0,174.0,26253.0
1,2024-05-16 09:05:00,178.5,177.0,178.5,177.0,10786.0
2,2024-05-16 09:10:00,177.0,177.0,178.5,176.5,9682.0
3,2024-05-16 09:15:00,176.5,176.5,177.0,176.0,5240.0
4,2024-05-16 09:20:00,176.0,175.5,176.5,175.5,4073.0


In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

# 禁用字型警告
matplotlib.rcParams['axes.unicode_minus'] = False

# 設置中文字型
zh_font = fm.FontProperties(fname='C:/Windows/Fonts/msjh.ttc')

# 使用函數畫圖
df

,時間,成交價,成交張數
0,2024-05-16 13:30:00,171.5,6083.0
1,2024-05-16 13:25:00,171.0,19.0
2,2024-05-16 13:24:55,171.0,127.0
3,2024-05-16 13:24:50,171.0,66.0
4,2024-05-16 13:24:45,171.0,14.0
...,...,...,...
3080,2024-05-16 09:00:20,175.0,994.0
3081,2024-05-16 09:00:15,175.0,242.0
3082,2024-05-16 09:00:10,175.0,241.0
3083,2024-05-16 09:00:05,174.5,402.0


# 抓檔案

# 104

In [ ]:
# 104的範例
my_service = Service(executable_path="./chromedriver-win64/chromedriver.exe") 
driver = webdriver.Chrome(options = my_options,service=my_service)

driver.get("https://www.104.com.tw/jobs/main/")

# 尋找網頁中的搜尋框
inputElement = driver.find_element(By.CSS_SELECTOR, 'input[data-gtm-index]')

# 在搜尋框中輸入文字
inputElement.send_keys("python")

# 睡個幾秒
sleep(2)

# 送出搜尋
inputElement.submit()

# 睡個幾秒
sleep(5)


# 關閉瀏覽器
driver.quit()
